In [1]:
from kafka import KafkaProducer
import threading
import json
import random
import time

from equipo.random_tweets import generate_random_tweets  

KAFKA_SERVER = 'ca342c443652:9093'
TOPIC_PREFIX = 'tweet-'

In [2]:
def worker(user_id: int, topic: str, num_messages: int):

    # Initialize the Kafka producer
    producer = KafkaProducer(
        bootstrap_servers=KAFKA_SERVER,
        value_serializer=lambda v: json.dumps(v).encode('utf-8')  # serialize data as JSON
    )

    print(f"[Thread-{user_id}] Iniciando, topic={topic}, mensajes={num_messages}")
    
    # Produce data to Kafka topic
    try:
        for tweet in generate_random_tweets(num_messages, user_id):
            # Send data to Kafka
            producer.send(topic, tweet)
            print(f"[Thread-{user_id}] Sent to {topic}: {tweet['tweet_id']}")

            time.sleep(2)

    except KeyboardInterrupt:
        print(f"[Thread-{user_id}] Interrumpido por el usuario.")
        
    finally:
        producer.close()
        print(f"[Thread-{user_id}] Producer cerrado.")

In [3]:
if __name__ == "__main__":
    NUM_MESSAGES = random.randint(100, 500)
    threads = []

    for user_id in range(1, 5):
        topic = f"{TOPIC_PREFIX}{user_id}"
        t = threading.Thread(
            target=worker,
            args=(user_id, topic, NUM_MESSAGES),
            name=f"Thread-{user_id}"
        )
        threads.append(t)
        t.start()

    # Wait for all threads to finish
    for t in threads:
        t.join()

    print("Todos los hilos han terminado.")


[Thread-1] Iniciando, topic=tweet-1, mensajes=398
[Thread-3] Iniciando, topic=tweet-3, mensajes=398
[Thread-2] Iniciando, topic=tweet-2, mensajes=398
[Thread-4] Iniciando, topic=tweet-4, mensajes=398
[Thread-2] Sent to tweet-2: a68824ad-2fc5-48fc-a21e-7fc38247e131
[Thread-3] Sent to tweet-3: 09f3adab-fdd5-43ce-b5cd-7bd6a7b3648d
[Thread-1] Sent to tweet-1: 3e90ca22-ae5f-4969-8ce0-a3daa0753824
[Thread-4] Sent to tweet-4: 722d0130-2e2f-415c-99a0-62202854f96b
[Thread-2] Sent to tweet-2: d8647f25-24d8-4ca9-bbb4-f74b7af40d6a
[Thread-3] Sent to tweet-3: 82dd6240-1a2a-4393-84ac-9151600b8be2
[Thread-1] Sent to tweet-1: 34a28583-a70d-4cad-b310-8f4fe275cf5c
[Thread-4] Sent to tweet-4: 8911d33e-0873-4e27-bbf2-320dccd53b08
[Thread-3] Sent to tweet-3: 9402f363-6fd2-4d51-bff5-97d5c3b685aa
[Thread-2] Sent to tweet-2: c2500e47-539b-4078-9bf6-5540976d4013
[Thread-1] Sent to tweet-1: ca9a0b19-33ed-4f12-a3db-7d1b659e6364
[Thread-4] Sent to tweet-4: 597e898f-870a-4571-9024-4cb24aadbacb
[Thread-3] Sent to t